In [ ]:
# Google colab version
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load df Google Colab (feature engineered, encoded, scaled/unscaled)
import pandas as pd
df_sampled_unscaled = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_sampled_unscaled.csv")
df_sampled_scaled = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_sampled_scaled.csv")

In [ ]:
import pandas as pd
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_final_2.csv")

# **Random Forest baseline nog runnen 15/11**

In [1]:
import numpy as np
import pandas as pd
import joblib
from google.colab import drive
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
# Load df Google Colab
drive.mount('/content/drive')
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_final_2.csv")

df_final = df_final.astype(np.int32)

Mounted at /content/drive


In [3]:
# Define feature and target
X = df_final.drop(columns=['Rating'])
y = df_final['Rating']

# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, random_state=42)

# Define the parameter grid for hyperparameter optimization
param_dist = {
    'n_estimators': [100, 200, 250, 300],
    'max_depth': [20, 30, 40, 50],
    'min_samples_split': [2,5,10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)


In [4]:
# Initialize the Randomized Search
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(),
                                   param_distributions=param_dist, n_iter=15,
                                   cv=inner_cv, scoring='accuracy')

# Define output lists
outer_scores = []
val_scores = []
best_params_list = []

# Perform nested CV
for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Save the best parameters for this fold
    best_params_list.append(random_search.best_params_)

    # Evaluate on the validation set
    val_predictions = best_model.predict(X_val_fold)
    val_acc = accuracy_score(y_val_fold, val_predictions)
    val_prec = precision_score(y_val_fold, val_predictions, average='weighted')
    val_rec = recall_score(y_val_fold, val_predictions, average='weighted')
    val_f1 = f1_score(y_val_fold, val_predictions, average='weighted')

    # Store validation metrics
    val_scores.append((val_acc, val_prec, val_rec, val_f1))

    # Store the outer fold score (accuracy)
    outer_scores.append(random_search.best_score_)

# Print the performance of the nested CV
print(f"Nested Cross-Validation Accuracy: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print validation set performance for each fold
val_scores = np.array(val_scores)
print("Validation Set Performance:")
print(f"Accuracy: {val_scores[:, 0].mean():.2f} ± {val_scores[:, 0].std():.2f}")
print(f"Precision: {val_scores[:, 1].mean():.2f} ± {val_scores[:, 1].std():.2f}")
print(f"Recall: {val_scores[:, 2].mean():.2f} ± {val_scores[:, 2].std():.2f}")
print(f"F1 Score: {val_scores[:, 3].mean():.2f} ± {val_scores[:, 3].std():.2f}")

# Print the best parameters found during hyperparameter tuning
print("Best parameters found during hyperparameter tuning:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i+1}: {params}")

# Define the best parameters found in nested CV
best_params = best_params_list[np.argmax(outer_scores)]

# Train the best model
best_model = RandomForestClassifier(**best_params, random_state=42)
best_model.fit(X_train, y_train)

# Evaluate on the test set
test_predictions = best_model.predict(X_test)
print("Test set performance for best performing model:")
print(f"Accuracy: {accuracy_score(y_test, test_predictions):.2f}")
print(f"Precision: {precision_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"Recall: {recall_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"F1 Score: {f1_score(y_test, test_predictions, average='weighted'):.2f}")

# Print the parameters for the best-performing model
print("Parameters for the best performing model:")
print(best_params)

# Save the best performing model
joblib_file = "/content/drive/My Drive/Thesis/Models/Baseline_RF.pkl"
joblib.dump(best_model, joblib_file)


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Nested Cross-Validation Accuracy: 0.41 ± 0.00
Validation Set Performance:
Accuracy: 0.42 ± 0.00
Precision: 0.42 ± 0.00
Recall: 0.42 ± 0.00
F1 Score: 0.39 ± 0.00
Best parameters found during hyperparameter tuning:
Fold 1: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 30}
Fold 2: {'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 40}
Fold 3: {'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 40}
Fold 4: {'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 40}
Fold 5: {'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 40}
Test set performance for best performing model:
Accuracy: 0.42
Precision: 0.43
Recall: 0.42
F1 Score: 0.40
Parameters for the best performing model:
{'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 40}


['/content/drive/My Drive/Thesis/Models/Baseline_RF.pkl']

# **XGBoost baseline 16/11**

In [ ]:
pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import KFold, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer

In [ ]:
# Load in the data
drive.mount('/content/drive')
df_final = pd.read_csv('/content/drive/My Drive/Thesis/Data/df_final_2.csv')

df_final = df_final.astype(np.int32)

Mounted at /content/drive


In [ ]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the "Rating" column
df_final['Rating'] = label_encoder.fit_transform(df_final['Rating'])

In [ ]:
# Define feature and target
X = df_final.drop(columns=['Rating'])
y = df_final['Rating']

# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, random_state=42)

# Define the parameter grid for tuning
param_dist = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7],
}

# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Define the scoring metric
f1_weighted_scorer = make_scorer(f1_score, average='weighted')

In [ ]:
# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=XGBClassifier(tree_method='hist', device='cuda',
                            eval_metric='merror'),
    param_distributions=param_dist,
    n_iter=15,
    cv=inner_cv,
    scoring=f1_weighted_scorer,
    n_jobs=-1
)

# Define output lists
outer_scores = []
val_scores = []
best_params_list = []

# Perform nested CV on the training data
for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Save the best parameters for this fold
    best_params_list.append(random_search.best_params_)

    # Evaluate on the validation set
    val_predictions = best_model.predict(X_val_fold)
    val_acc = accuracy_score(y_val_fold, val_predictions)
    val_prec = precision_score(y_val_fold, val_predictions, average='weighted')
    val_rec = recall_score(y_val_fold, val_predictions, average='weighted')
    val_f1 = f1_score(y_val_fold, val_predictions, average='weighted')

    # Store validation metrics
    val_scores.append((val_acc, val_prec, val_rec, val_f1))

    # Store the outer fold score (f1_weighted)
    outer_scores.append(random_search.best_score_)

# Print the performance of the nested CV
print(f"Nested Cross-Validation F1 Score: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print validation set performance for each fold
val_scores = np.array(val_scores)
print("Validation Set Performance:")
print(f"Accuracy: {val_scores[:, 0].mean():.2f} ± {val_scores[:, 0].std():.2f}")
print(f"Precision: {val_scores[:, 1].mean():.2f} ± {val_scores[:, 1].std():.2f}")
print(f"Recall: {val_scores[:, 2].mean():.2f} ± {val_scores[:, 2].std():.2f}")
print(f"F1 Score: {val_scores[:, 3].mean():.2f} ± {val_scores[:, 3].std():.2f}")

# Print the best parameters found during hyperparameter tuning
print("Best parameters found during hyperparameter tuning:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i+1}: {params}")

# Define the best parameters found in nested CV
best_params = best_params_list[np.argmax(outer_scores)]

# Train best model on the entire training data
best_model = XGBClassifier(**best_params, tree_method='hist',
                           device='cuda', eval_metric='merror',
                           random_state=42)
best_model.fit(X_train, y_train)

# Evaluate on the test set
test_predictions = best_model.predict(X_test)
print("Test set performance for best performing model:")
print(f"Accuracy: {accuracy_score(y_test, test_predictions):.2f}")
print(f"Precision: {precision_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"Recall: {recall_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"F1 Score: {f1_score(y_test, test_predictions, average='weighted'):.2f}")

# Print the parameters for the best-performing model
print("Parameters for the best performing model:")
print(best_params)

# Save the best performing model
joblib_file = "/content/drive/My Drive/Thesis/Models/Baseline_XGB.pkl"
joblib.dump(best_model, joblib_file)


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [11:46:48] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Nested Cross-Validation F1 Score: 0.41 ± 0.00
Validation Set Performance:
Accuracy: 0.44 ± 0.00
Precision: 0.43 ± 0.00
Recall: 0.44 ± 0.00
F1 Score: 0.42 ± 0.00
Best parameters found during hyperparameter tuning:
Fold 1: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.2}
Fold 2: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.1}
Fold 3: {'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.2}
Fold 4: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.1}
Fold 5: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.2}
Test set performance for best performing model:
Accuracy: 0.44
Precision: 0.44
Recall: 0.44
F1 Score: 0.43
Parameters for the best performing model:
{'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.2}


['/content/drive/My Drive/Thesis/Models/Baseline_XGB.pkl']

# **Collaborative Filtering, matrix factorizaiton**

In [ ]:
# Google colab version
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_final_2.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class ImputationMatrixFactorization:
    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def impute_missing_values(self):
        imputer = SimpleImputer(strategy='mean')
        self.R_imputed = imputer.fit_transform(self.R)

    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R_imputed[np.where(self.R_imputed != 0)])

        self.samples = [
            (i, j, self.R_imputed[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            if (i+1) % 10 == 0:
                print(f"Iteration: {i+1}; error = {mse:.4f}")

    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += (self.R[x, y] - predicted[x, y]) ** 2
        return np.sqrt(error)

    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)



In [ ]:
R_df = df_final.pivot(index='UserID', columns='MovieID', values='Rating')
R = R_df.values

imf = ImputationMatrixFactorization(R, K=2, alpha=0.01, beta=0.01, iterations=100)
imf.impute_missing_values()
imf.train()
predicted_matrix = imf.full_matrix()
print(predicted_matrix)

# Evaluation
def get_true_pred_ratings(df, predicted_df):
    true_ratings = []
    predicted_ratings = []
    for row in df.itertuples():
        user_id = row.UserID
        movie_id = row.MovieID
        true_ratings.append(row.Rating)
        predicted_ratings.append(predicted_df.loc[user_id, movie_id])
    return np.array(true_ratings), np.array(predicted_ratings)

true_ratings, predicted_ratings = get_true_pred_ratings(df_final, pd.DataFrame(predicted_matrix, columns=R_df.columns, index=R_df.index))

precision = precision_score(true_ratings, predicted_ratings.round(), average='weighted', zero_division=0)
recall = recall_score(true_ratings, predicted_ratings.round(), average='weighted', zero_division=0)
f1 = f1_score(true_ratings, predicted_ratings.round(), average='weighted', zero_division=0)
accuracy = accuracy_score(true_ratings, predicted_ratings.round())

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

Iteration: 10; error = nan
Iteration: 20; error = nan
Iteration: 30; error = nan
Iteration: 40; error = nan
Iteration: 50; error = nan
Iteration: 60; error = nan
Iteration: 70; error = nan
Iteration: 80; error = nan
Iteration: 90; error = nan
Iteration: 100; error = nan
[[4.46010186 3.60829331 3.43042852 ... 4.32580485 2.65673402 3.87182357]
 [4.14812481 3.34335054 3.14951091 ... 3.96385389 2.55786762 3.69507351]
 [4.27883658 3.44457187 3.23395441 ... 3.92351679 3.35727072 4.12188705]
 ...
 [4.46996054 4.18968623 3.91941883 ... 4.49668633 2.21048452 4.07896634]
 [4.19810411 3.50903944 3.27162849 ... 3.85862852 3.26002517 4.13844656]
 [3.44117644 1.93717885 1.8060465  ... 2.67968091 3.91627503 3.46236176]]
Precision: 0.5177
Recall: 0.4668
F1 Score: 0.4472
Accuracy: 0.4668


# OLD, NOT WORKING

## RF, HPT, (-2 features) -> Accuracy 0.46 (wrong data split)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

In [ ]:
# Separate features and target
X = df_final.drop(columns=['Rating'])
y = df_final['Rating']

# Perform stratified sampling for 40% of the data for increased efficiency
X_sampled, _, y_sampled, _ = train_test_split(X, y, test_size=0.6, stratify=y)


In [ ]:
def evaluate_performance(y_true, y_pred):
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.2f}")
    print(f"Precision: {precision_score(y_true, y_pred, average='macro'):.2f}")
    print(f"Recall: {recall_score(y_true, y_pred, average='macro'):.2f}")
    print(f"F1 Score: {f1_score(y_true, y_pred, average='macro'):.2f}")


In [ ]:
# Define the parameter grid for tuning
param_dist = {
    'n_estimators': [10, 50, 100],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(),
                                   param_distributions=param_dist, n_iter=10,
                                   cv=inner_cv, scoring='accuracy',
                                   random_state=42)

# Perform nested cross-validation
nested_scores = cross_val_score(random_search, X_sampled, y_sampled,
                                cv=outer_cv, scoring='accuracy')

# Print the performance of the nested cross-validation
print(f"Nested Cross-Validation Accuracy: {nested_scores.mean():.2f} ± {nested_scores.std():.2f}")

Nested Cross-Validation Accuracy: 0.46 ± 0.00


In [ ]:
# Split the data into training, validation, and test sets (60%, 20%, 20%)
X_train, X_temp, y_train, y_temp = train_test_split(X_sampled, y_sampled, test_size=0.4, stratify=y_sampled)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp)

# Fit the best model found by the random search on the full training set
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_

# Evaluate the best model on the validation set
val_predictions = best_model.predict(X_val)
print("Validation Set Performance:")
evaluate_performance(y_val, val_predictions)

# Evaluate the best model on the test set
test_predictions = best_model.predict(X_test)
print("Test Set Performance:")
evaluate_performance(y_test, test_predictions)


Validation Set Performance:
Accuracy: 0.46
Precision: 0.47
Recall: 0.38
F1 Score: 0.40
Test Set Performance:
Accuracy: 0.45
Precision: 0.47
Recall: 0.38
F1 Score: 0.40


## RF, HPT, (-2 features) -> Accuracy 0.46 (no val scores)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

In [ ]:
# Load in the dataframe
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_sampled_unscaled.csv")
df_final = df_final.drop(columns=['Dev_movie_avg',	'Total_ratings_per_user'])

In [ ]:
# Separate features and target
X = df_final.drop(columns=['Rating'])
y = df_final['Rating']

In [ ]:
# Split sampled data into 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)


In [ ]:
# Evaluation function
def evaluate_performance(y_true, y_pred):
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.2f}")
    print(f"Precision: {precision_score(y_true, y_pred, average='macro'):.2f}")
    print(f"Recall: {recall_score(y_true, y_pred, average='macro'):.2f}")
    print(f"F1 Score: {f1_score(y_true, y_pred, average='macro'):.2f}")


In [ ]:
# Define the parameter grid for tuning

param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6]
}

# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(),
                                   param_distributions=param_dist, n_iter=15,
                                   cv=inner_cv, scoring='accuracy',
                                   random_state=42)

# Perform nested cross-validation and store best parameters for each fold
nested_scores = []
best_params = []
best_models = []

for train_idx, test_idx in outer_cv.split(X_train, y_train):
    random_search.fit(X_train.iloc[train_idx], y_train.iloc[train_idx])
    nested_scores.append(random_search.best_score_)
    best_params.append(random_search.best_params_)
    best_model = random_search.best_estimator_
    best_models.append(best_model)

# Print the performance of the nested cross-validation
print(f"Nested Cross-Validation Accuracy: {np.mean(nested_scores):.2f} ± {np.std(nested_scores):.2f}")

# Print the best parameters found for each fold
print("Best parameters for each fold:")
for params in best_params:
    print(params)

# Use the best model found in the outer loop
final_best_model = best_models[-1]

# Evaluate the best model on the test set
test_predictions = final_best_model.predict(X_test)
print("Test Set Performance:")
evaluate_performance(y_test, test_predictions)

In [ ]:
# Evaluate the best model on the test set
test_predictions = final_best_model.predict(X_test)
print("Test Set Performance:")
evaluate_performance(y_test, test_predictions)

Test Set Performance:
Accuracy: 0.46
Precision: 0.48
Recall: 0.37
F1 Score: 0.39


## 29/10 Random Forest with hyperparameter tuning (new sample_df)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Define feature and target
X = df_final.drop('Rating', axis=1)
y = df_final['Rating']

In [ ]:
# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
# Define the parameter grid for hyperparameter optimization
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(),
                                   param_distributions=param_dist, n_iter=15,
                                   cv=inner_cv, scoring='accuracy',
                                   random_state=42)

# Perform nested cross-validation
outer_scores = []
val_scores = []

for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Evaluate on the validation set
    val_predictions = best_model.predict(X_val_fold)
    val_acc = accuracy_score(y_val_fold, val_predictions)
    val_prec = precision_score(y_val_fold, val_predictions, average='weighted')
    val_rec = recall_score(y_val_fold, val_predictions, average='weighted')
    val_f1 = f1_score(y_val_fold, val_predictions, average='weighted')

    # Append validation metrics
    val_scores.append((val_acc, val_prec, val_rec, val_f1))

    # Store the outer fold score (accuracy)
    outer_scores.append(random_search.best_score_)

# Print the performance of the nested cross-validation
print(f"Nested Cross-Validation Accuracy: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print validation set performance for each fold
val_scores = np.array(val_scores)
print("Validation Set Performance:")
print(f"Accuracy: {val_scores[:, 0].mean():.2f} ± {val_scores[:, 0].std():.2f}")
print(f"Precision: {val_scores[:, 1].mean():.2f} ± {val_scores[:, 1].std():.2f}")
print(f"Recall: {val_scores[:, 2].mean():.2f} ± {val_scores[:, 2].std():.2f}")
print(f"F1 Score: {val_scores[:, 3].mean():.2f} ± {val_scores[:, 3].std():.2f}")

# Fit the best model found on the entire training set
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_

# Evaluate on the test set
test_predictions = best_model.predict(X_test)
print("Test Set Performance:")
print(f"Accuracy: {accuracy_score(y_test, test_predictions):.2f}")
print(f"Precision: {precision_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"Recall: {recall_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"F1 Score: {f1_score(y_test, test_predictions, average='weighted'):.2f}")


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Nested Cross-Validation Accuracy: 0.41 ± 0.00
Validation Set Performance:
Accuracy: 0.41 ± 0.00
Precision: 0.42 ± 0.00
Recall: 0.41 ± 0.00
F1 Score: 0.38 ± 0.00
Test Set Performance:
Accuracy: 0.42
Precision: 0.43
Recall: 0.42
F1 Score: 0.39


## 30/10 Random Forest with hyperparameter tuning, sample_df_3010



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
import pandas as pd
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/sample_df_3010.csv")

In [ ]:
# Define feature and target
X = df_final.drop('Rating', axis=1)
y = df_final['Rating']

In [ ]:
# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, random_state=42)

In [ ]:
# Define the parameter grid for hyperparameter optimization
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(),
                                   param_distributions=param_dist, n_iter=15,
                                   cv=inner_cv, scoring='accuracy',
                                   random_state=42)



# Perform nested cross-validation
outer_scores = []
val_scores = []

for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Evaluate on the validation set
    val_predictions = best_model.predict(X_val_fold)
    val_acc = accuracy_score(y_val_fold, val_predictions)
    val_prec = precision_score(y_val_fold, val_predictions, average='weighted')
    val_rec = recall_score(y_val_fold, val_predictions, average='weighted')
    val_f1 = f1_score(y_val_fold, val_predictions, average='weighted')

    # Append validation metrics
    val_scores.append((val_acc, val_prec, val_rec, val_f1))

    # Store the outer fold score (accuracy)
    outer_scores.append(random_search.best_score_)

# Print the performance of the nested cross-validation
print(f"Nested Cross-Validation Accuracy: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print validation set performance for each fold
val_scores = np.array(val_scores)
print("Validation Set Performance:")
print(f"Accuracy: {val_scores[:, 0].mean():.2f} ± {val_scores[:, 0].std():.2f}")
print(f"Precision: {val_scores[:, 1].mean():.2f} ± {val_scores[:, 1].std():.2f}")
print(f"Recall: {val_scores[:, 2].mean():.2f} ± {val_scores[:, 2].std():.2f}")
print(f"F1 Score: {val_scores[:, 3].mean():.2f} ± {val_scores[:, 3].std():.2f}")

# Fit the best model found on the entire training set
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_

# Evaluate on the test set
test_predictions = best_model.predict(X_test)
print("Test Set Performance:")
print(f"Accuracy: {accuracy_score(y_test, test_predictions):.2f}")
print(f"Precision: {precision_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"Recall: {recall_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"F1 Score: {f1_score(y_test, test_predictions, average='weighted'):.2f}")


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Nested Cross-Validation Accuracy: 0.40 ± 0.00
Validation Set Performance:
Accuracy: 0.41 ± 0.00
Precision: 0.42 ± 0.00
Recall: 0.41 ± 0.00
F1 Score: 0.38 ± 0.00
Test Set Performance:
Accuracy: 0.41
Precision: 0.42
Recall: 0.41
F1 Score: 0.38


##  27/10 Random Forest with hyperparameter tuning (-2 features) -> Accuracy ?

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Load df Google Colab (feature engineered, encoded, scaled/unscaled)
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_sampled_unscaled.csv")
df_final = df_final.drop(columns=['Dev_movie_avg',	'Total_ratings_per_user'])

In [ ]:
# Define feature and target
X = df_final.drop('Rating', axis=1)
y = df_final['Rating']

In [ ]:
# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
# Define the parameter grid for hyperparameter optimization
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(),
                                   param_distributions=param_dist, n_iter=15,
                                   cv=inner_cv, scoring='accuracy',
                                   random_state=42)

# Perform nested cross-validation
outer_scores = []
val_scores = []

for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Evaluate on the validation set
    val_predictions = best_model.predict(X_val_fold)
    val_acc = accuracy_score(y_val_fold, val_predictions)
    val_prec = precision_score(y_val_fold, val_predictions, average='weighted')
    val_rec = recall_score(y_val_fold, val_predictions, average='weighted')
    val_f1 = f1_score(y_val_fold, val_predictions, average='weighted')

    # Append validation metrics
    val_scores.append((val_acc, val_prec, val_rec, val_f1))

    # Store the outer fold score (accuracy)
    outer_scores.append(random_search.best_score_)

# Print the performance of the nested cross-validation
print(f"Nested Cross-Validation Accuracy: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print validation set performance for each fold
val_scores = np.array(val_scores)
print("Validation Set Performance:")
print(f"Accuracy: {val_scores[:, 0].mean():.2f} ± {val_scores[:, 0].std():.2f}")
print(f"Precision: {val_scores[:, 1].mean():.2f} ± {val_scores[:, 1].std():.2f}")
print(f"Recall: {val_scores[:, 2].mean():.2f} ± {val_scores[:, 2].std():.2f}")
print(f"F1 Score: {val_scores[:, 3].mean():.2f} ± {val_scores[:, 3].std():.2f}")

# Fit the best model found on the entire training set
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_

# Evaluate on the test set
test_predictions = best_model.predict(X_test)
print("Test Set Performance:")
print(f"Accuracy: {accuracy_score(y_test, test_predictions):.2f}")
print(f"Precision: {precision_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"Recall: {recall_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"F1 Score: {f1_score(y_test, test_predictions, average='weighted'):.2f}")

Nested Cross-Validation Accuracy: 0.46 ± 0.00
Validation Set Performance:
Accuracy: 0.46 ± 0.00
Precision: 0.47 ± 0.00
Recall: 0.46 ± 0.00
F1 Score: 0.44 ± 0.00
Test Set Performance:
Accuracy: 0.46
Precision: 0.47
Recall: 0.46
F1 Score: 0.45


## Gradient boosting with hyperparameter tuning (-2 features) -> Accuracy 0.46


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

In [ ]:
# Load df Google Colab (feature engineered, encoded, scaled/unscaled)
df_sampled_unscaled = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_sampled_unscaled.csv")
df_sampled_scaled = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_sampled_scaled.csv")

In [ ]:
df_final = df_sampled_unscaled.copy()
df_final = df_final.drop(columns=['Dev_movie_avg',	'Total_ratings_per_user'])

In [ ]:
# Separate features and target
X = df_final.drop(columns=['Rating'])
y = df_final['Rating']

In [ ]:
# Split data into 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
# Define evaluation function
def evaluate_performance(y_true, y_pred):
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.2f}")
    print(f"Precision: {precision_score(y_true, y_pred, average='macro'):.2f}")
    print(f"Recall: {recall_score(y_true, y_pred, average='macro'):.2f}")
    print(f"F1 Score: {f1_score(y_true, y_pred, average='macro'):.2f}")

In [ ]:
from joblib import Parallel, delayed

# Define the parameter grid for tuning
param_dist = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 7, 10, 20]
}

# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=GradientBoostingClassifier(),
    param_distributions=param_dist,
    n_iter=15,
    cv=inner_cv,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1  # Utilize all available cores for parallel processing
)

# Perform nested cross-validation and store best parameters for each fold
nested_scores = []
best_params = []
best_models = []

def fit_and_evaluate(train_idx, test_idx):
    random_search.fit(X_train.iloc[train_idx], y_train.iloc[train_idx])
    best_model = random_search.best_estimator_
    return random_search.best_score_, random_search.best_params_, best_model

results = Parallel(n_jobs=-1)(delayed(fit_and_evaluate)(train_idx, test_idx) for train_idx, test_idx in outer_cv.split(X_train, y_train))

for score, params, model in results:
    nested_scores.append(score)
    best_params.append(params)
    best_models.append(model)

# Print the performance of the nested cross-validation
print(f"Nested Cross-Validation Accuracy: {np.mean(nested_scores):.2f} ± {np.std(nested_scores):.2f}")

# Print the best parameters found for each fold
print("Best parameters for each fold:")
for params in best_params:
    print(params)

# Use the best model found in the outer loop
final_best_model = best_models[-1]

# Evaluate the best model on the test set
test_predictions = final_best_model.predict(X_test)
print("Test Set Performance:")
evaluate_performance(y_test, test_predictions)


Nested Cross-Validation Accuracy: 0.46 ± 0.00
Best parameters for each fold:
{'n_estimators': 50, 'max_depth': 7, 'learning_rate': 0.1}
{'n_estimators': 50, 'max_depth': 7, 'learning_rate': 0.1}
{'n_estimators': 50, 'max_depth': 7, 'learning_rate': 0.1}
{'n_estimators': 50, 'max_depth': 7, 'learning_rate': 0.1}
{'n_estimators': 50, 'max_depth': 7, 'learning_rate': 0.1}
Test Set Performance:
Accuracy: 0.46
Precision: 0.46
Recall: 0.38
F1 Score: 0.40


## **BASELINE RANDOM FOREST 10/11, df_final_2**

> Good dataframe, not good that gridcv is ran twice!!



In [ ]:
# Google colab version
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load df Google Colab (feature engineered, encoded, scaled/unscaled)
import pandas as pd
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_final_2.csv")

pd.set_option('display.max_columns', None)
df_final.head()

,UserID,MovieID,Rating,Age,Year,Month,Day,Hour,Release_year,Time_release_to_rating,Total_ratings_per_movie,Total_ratings_per_user,Female,Male,Academic/educator,Artist,Clerical/admin,College/grad student,Customer service,Doctor/health care,Executive/managerial,Farmer,Homemaker,K-12 student,Lawyer,Other or not specified,Programmer,Retired,Sales/marketing,Scientist,Self-employed,Technician/engineer,Tradesman/craftsman,Unemployed,Writer,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Favourite_Action,Favourite_Adventure,Favourite_Animation,Favourite_Children's,Favourite_Comedy,Favourite_Crime,Favourite_Documentary,Favourite_Drama,Favourite_Fantasy,Favourite_Film-Noir,Favourite_Horror,Favourite_Musical,Favourite_Mystery,Favourite_Romance,Favourite_Sci-Fi,Favourite_Thriller,Favourite_War,Favourite_Western
0,5616,3590,3,4,2000,5,24,2,1974,26,85,130,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,4060,21,4,2,2000,8,5,15,1995,5,1356,256,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,1125,3273,2,1,2000,11,22,16,2000,0,577,580,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
3,3410,585,4,3,2000,8,27,22,1995,5,418,731,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
4,3675,1374,4,3,2000,8,15,18,1982,18,1448,849,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Define feature and target
X = df_final.drop('Rating', axis=1)
y = df_final['Rating']

# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, random_state=42)

# Define the parameter grid for hyperparameter optimization
param_dist = {
    'n_estimators': [100, 200, 250, 300],
    'max_depth': [20, 30, 40, 50],
    'min_samples_split': [2,5,10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Turning booleans into numerical values
#df_final = np.nan_to_num(df_final)
df_final.astype(np.int32)

array([[5616, 3590,    3, ...,    0,    0,    1],
       [4060,   21,    4, ...,    0,    0,    1],
       [1125, 3273,    2, ...,    0,    0,    0],
       ...,
       [1100, 3525,    1, ...,    0,    0,    0],
       [1971, 2355,    4, ...,    0,    1,    0],
       [5136, 1429,    4, ...,    0,    0,    0]], dtype=int32)

In [ ]:
# Initialize the Ranodom Search
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(),
                                   param_distributions=param_dist, n_iter=15,
                                   cv=inner_cv, scoring='accuracy',
                                   random_state=42)

# Define output lists
outer_scores = []
val_scores = []
best_params_list = []

# Perform nested cross-validation
for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Save the best parameters for this fold
    best_params_list.append(random_search.best_params_)

    # Evaluate on the validation set
    val_predictions = best_model.predict(X_val_fold)
    val_acc = accuracy_score(y_val_fold, val_predictions)
    val_prec = precision_score(y_val_fold, val_predictions, average='weighted')
    val_rec = recall_score(y_val_fold, val_predictions, average='weighted')
    val_f1 = f1_score(y_val_fold, val_predictions, average='weighted')

    # Append validation metrics
    val_scores.append((val_acc, val_prec, val_rec, val_f1))

    # Store the outer fold score (accuracy)
    outer_scores.append(random_search.best_score_)

# Print the performance of the nested cross-validation
print(f"Nested Cross-Validation Accuracy: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print validation set performance for each fold
val_scores = np.array(val_scores)
print("Validation Set Performance:")
print(f"Accuracy: {val_scores[:, 0].mean():.2f} ± {val_scores[:, 0].std():.2f}")
print(f"Precision: {val_scores[:, 1].mean():.2f} ± {val_scores[:, 1].std():.2f}")
print(f"Recall: {val_scores[:, 2].mean():.2f} ± {val_scores[:, 2].std():.2f}")
print(f"F1 Score: {val_scores[:, 3].mean():.2f} ± {val_scores[:, 3].std():.2f}")

# Print the best parameters found during hyperparameter tuning
print("Best parameters found during hyperparameter tuning:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i+1}: {params}")

# Fit the best model found on the entire training set
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_

# Evaluate on the test set
test_predictions = best_model.predict(X_test)
print("Test Set Performance:")
print(f"Accuracy: {accuracy_score(y_test, test_predictions):.2f}")
print(f"Precision: {precision_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"Recall: {recall_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"F1 Score: {f1_score(y_test, test_predictions, average='weighted'):.2f}")

# Print the best parameters of the best-performing model
print("Best Parameters of the Best-Performing Model:")
print(random_search.best_params_)

Nested Cross-Validation Accuracy: 0.41 ± 0.00
Validation Set Performance:
Accuracy: 0.42 ± 0.00
Precision: 0.42 ± 0.00
Recall: 0.42 ± 0.00
F1 Score: 0.39 ± 0.00
Best parameters found during hyperparameter tuning:
Fold 1: {'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 40}
Fold 2: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 30}
Fold 3: {'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 40}
Fold 4: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 30}
Fold 5: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 30}
Test Set Performance:
Accuracy: 0.42
Precision: 0.43
Recall: 0.42
F1 Score: 0.40
Best Parameters of the Best-Performing Model:
{'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 40}


## Gradient boosting, goede dataframe, goede code, nog runnen 15/11

In [ ]:
import pandas as pd
import numpy as np
import joblib
from google.colab import drive
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV

In [ ]:
# Load df Google Colab (feature engineered, encoded, scaled/unscaled)
drive.mount('/content/drive')
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_final_2.csv")

df_final = df_final.astype(np.int32)

In [ ]:
# Define feature and target
X = df_final.drop(columns=['Rating'])
y = df_final['Rating']

# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, random_state=42)
# Define the parameter grid for tuning
param_dist = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 7, 10, 20]
}

# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=GradientBoostingClassifier(),
    param_distributions=param_dist,
    n_iter=15,
    cv=inner_cv,
    scoring='accuracy',
    n_jobs=-1  # Utilize all available cores for parallel processing
)

# Define output lists
outer_scores = []
val_scores = []
best_params_list = []

# Perform nested CV
for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Save the best parameters for this fold
    best_params_list.append(random_search.best_params_)

    # Evaluate on the validation set
    val_predictions = best_model.predict(X_val_fold)
    val_acc = accuracy_score(y_val_fold, val_predictions)
    val_prec = precision_score(y_val_fold, val_predictions, average='weighted')
    val_rec = recall_score(y_val_fold, val_predictions, average='weighted')
    val_f1 = f1_score(y_val_fold, val_predictions, average='weighted')

    # Store validation metrics
    val_scores.append((val_acc, val_prec, val_rec, val_f1))

    # Store the outer fold score (accuracy)
    outer_scores.append(random_search.best_score_)

# Print the performance of the nested CV
print(f"Nested Cross-Validation Accuracy: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print validation set performance for each fold
val_scores = np.array(val_scores)
print("Validation Set Performance:")
print(f"Accuracy: {val_scores[:, 0].mean():.2f} ± {val_scores[:, 0].std():.2f}")
print(f"Precision: {val_scores[:, 1].mean():.2f} ± {val_scores[:, 1].std():.2f}")
print(f"Recall: {val_scores[:, 2].mean():.2f} ± {val_scores[:, 2].std():.2f}")
print(f"F1 Score: {val_scores[:, 3].mean():.2f} ± {val_scores[:, 3].std():.2f}")

# Print the best parameters found during hyperparameter tuning
print("Best parameters found during hyperparameter tuning:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i+1}: {params}")

# Define the best parameters found in nested CV
best_params = best_params_list[np.argmax(outer_scores)]

# Train best model
best_model = GradientBoostingClassifier(**best_params, random_state=42)
best_model.fit(X_train, y_train)

# Evaluate on the test set
test_predictions = best_model.predict(X_test)
print("Test set performance for best performing model:")
print(f"Accuracy: {accuracy_score(y_test, test_predictions):.2f}")
print(f"Precision: {precision_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"Recall: {recall_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"F1 Score: {f1_score(y_test, test_predictions, average='weighted'):.2f}")

# Print the parameters for the best-performing model
print("Parameters for the best performing model:")
print(best_params)

# Save the best performing model
joblib_file = "/content/drive/My Drive/Thesis/Models/Baseline_GB.pkl"
joblib.dump(best_model, joblib_file)